Create retriever

In [219]:
from haystack.document_stores.elasticsearch import ElasticsearchDocumentStore

In [220]:
document_store = ElasticsearchDocumentStore(host='localhost', username='', password='', index='squad_docs')

Make the google query

In [230]:
from googlesearch import search

max_number = 10
valid_url = []

# to search
query = "Trigonometry"

for j in search(query, num=20, start = 1, stop=15, lang = 'en'):
    print(j)
    valid_url.append(j)

https://en.wikipedia.org/wiki/Trigonometry
https://www.britannica.com/science/trigonometry
https://www.mathsisfun.com/algebra/trigonometry.html
https://www.youtube.com/watch?v=PUB0TaZ7bhA
https://www.cuemath.com/trigonometry/
https://byjus.com/maths/trigonometry/
https://www.skillsyouneed.com/num/trigonometry.html
https://www2.clarku.edu/faculty/djoyce/trig/
https://mathworld.wolfram.com/Trigonometry.html
https://www.geogebra.org/t/trigonometry?lang=en
https://www.govst.edu/uploadedFiles/Academics/Colleges_and_Programs/CAS/Trigonometry_Short_Course_Tutorial_Lauren_Johnson.pdf
https://www.livescience.com/51026-what-is-trigonometry.html
https://www.wolframalpha.com/examples/mathematics/trigonometry
https://www.sfu.ca/math-coursenotes/Math%20157%20Course%20Notes/sec_Trigonometry.html
https://www.omnicalculator.com/math/trigonometry


In [231]:
valid_url

['https://en.wikipedia.org/wiki/Trigonometry',
 'https://www.britannica.com/science/trigonometry',
 'https://www.mathsisfun.com/algebra/trigonometry.html',
 'https://www.youtube.com/watch?v=PUB0TaZ7bhA',
 'https://www.cuemath.com/trigonometry/',
 'https://byjus.com/maths/trigonometry/',
 'https://www.skillsyouneed.com/num/trigonometry.html',
 'https://www2.clarku.edu/faculty/djoyce/trig/',
 'https://mathworld.wolfram.com/Trigonometry.html',
 'https://www.geogebra.org/t/trigonometry?lang=en',
 'https://www.govst.edu/uploadedFiles/Academics/Colleges_and_Programs/CAS/Trigonometry_Short_Course_Tutorial_Lauren_Johnson.pdf',
 'https://www.livescience.com/51026-what-is-trigonometry.html',
 'https://www.wolframalpha.com/examples/mathematics/trigonometry',
 'https://www.sfu.ca/math-coursenotes/Math%20157%20Course%20Notes/sec_Trigonometry.html',
 'https://www.omnicalculator.com/math/trigonometry']

In [232]:
import urllib
from bs4 import BeautifulSoup
from langdetect import detect
import requests

import re

min_words = 400

pages = []

for url in valid_url:
    
    innerHtml = requests.get(url).content
    
    ##Use BeautifulSoup to convert the html file to plain text
    soup = BeautifulSoup(innerHtml, "html.parser")
    html_text = soup.get_text()
    html_text = re.sub('(?<=\n)\s+\n', '\n', html_text)
    html_text = re.sub('\s{2,}', '\n', html_text)
    html_text = re.sub('\[.]', '', html_text)

    
    #Detect if the text is in english
    language = detect(html_text)
    
    if len(html_text.split(" ")) > min_words and language == 'en':
          ##Add the plain text to the rawdata list
          pages.append(html_text)



WARNING - bs4.dammit -  Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [233]:
pages[2]

'\nTrigonometry\nIntroduction to Trigonometry\nTrigonometry (from Greek trigonon "triangle" + metron "measure")\nWant to learn Trigonometry? Here is a quick summary.\nFollow the links for more, or go to Trigonometry Index\nTrigonometry ... is all about triangles.\nTrigonometry helps us find angles and distances, and is used a lot in science, engineering, video games, and more!\nRight-Angled Triangle\nThe triangle of most interest is the right-angled triangle. The right angle is shown by the little box in the corner:\nAnother angle is often labeled θ, and the three sides are then called:\nAdjacent: adjacent (next to) the angle θ\nOpposite: opposite the angle θ\nand the longest side is the Hypotenuse\nWhy a Right-Angled Triangle?\nWhy is this triangle so important?\nImagine we can measure along and up but want to know the direct distance and angle:\nTrigonometry can find that missing angle and distance.\nOr maybe we have a distance and angle and need to "plot the dot" along and up:\nQues

Paragraph selector

In [234]:
min_words_per_paragraph = 40

contexts = []

for page in pages:
    
    paragraphs = page.split("\n")
    
    for paragraph in paragraphs:
        
        size = len(paragraph.split(" "))
    
        if size >= min_words_per_paragraph:
            contexts.append(paragraph)
    


In [235]:
len(contexts)

96

In [236]:
contexts[50]

'In Diagram ii, we have rotated the radius further in an anti-clockwise direction, past the vertical (y axis) into the next quadrant. Here θ is an obtuse angle, between 90° and 180°. The reference angle alpha α is equal to 180° − θ, and is the acute angle within the right-angled triangle.'

In [237]:
squad_docs = []

for context in contexts:
    squad_docs.append({
        'content': context
    })

Put data in

In [238]:
document_store.write_documents(squad_docs)

Retrive

In [239]:
from haystack.retriever.sparse import BM25Retriever

retriever = BM25Retriever(document_store)

In [188]:
retriever.retrieve("When was UNAM created?")

[<Document: {'content': 'The National Autonomous University of Mexico was officially established in 1910,[279] and the university became one of the most important institutes of higher learning in Mexico.[280] UNAM provides world class education in science, medicine, and engineering.[281] Many scientific institutes and new institutes of higher learning, such as National Polytechnic Institute (founded in 1936),[282] were established during the first half of the 20th century. Most of the new research institutes were created within UNAM. Twelve institutes were integrated into UNAM from 1929 to 1973.[283] In 1959, the Mexican Academy of Sciences was created to coordinate scientific efforts between academics.', 'content_type': 'text', 'score': 0.8534248064768555, 'meta': {}, 'embedding': None, 'id': '8067da99344acc9ddd67863d032e0d24'}>,
 <Document: {'content': 'In 1995, the Mexican chemist Mario J. Molina shared the Nobel Prize in Chemistry with Paul J. Crutzen and F. Sherwood Rowland for th

In [211]:


res = requests.post('http://localhost:9200/squad_docs/_delete_by_query',
                    json={
                        'query': {
                            'match_all': {}
                        }
                    })

res.json()

{'took': 1,
 'timed_out': False,
 'total': 0,
 'deleted': 0,
 'batches': 0,
 'version_conflicts': 0,
 'noops': 0,
 'retries': {'bulk': 0, 'search': 0},
 'throttled_millis': 0,
 'requests_per_second': -1.0,
 'throttled_until_millis': 0,
 'failures': []}